In [1]:
import numpy as np

In [2]:
np.set_printoptions(precision=12, suppress=True)

In [3]:
consider_distort = True # flag whether to consider distortion
zoomed_coords = True # whether to divide coordinates by 5 or not (zoomed coords or not)

folder_name = 'data/real_stars_3072/'

fnames = [ 'coords_mesh_center' ]

# stars from center of pictures only
# fnames = [
#     '20160218_201934_unmod_stars_coords',
#     '20160219_021934_unmod_stars_coords',
#     '20160323_031848_unmod_stars_coords',
#     '20160329_043931_unmod_stars_coords',
#     '20160507_000119_unmod_stars_coords'
# ]

# stars from whole pictures
# fnames = [
#     '20151016_221239_stars_coords',
#     '20160218_201934_stars_coords',
#     '20160219_021934_stars_coords',
#     '20160323_031848_stars_coords',
#     '20160329_043931_stars_coords',
#     '20160507_000119_stars_coords'
# ]

In [4]:
if consider_distort:
    width = 2048
    height = 1536
    x_c = width /2
    y_c = height /2
    
    print 'Check this!'
    print 'Image %r x %r' % (width, height)
    print 'X_center: %r, Y_center: %r' % (x_c, y_c)

Check this!
Image 2048 x 1536
X_center: 1024, Y_center: 768


In [5]:
import os

In [6]:
coords_list = []
for fname in fnames:
    piece = np.loadtxt(folder_name + os.sep + fname)
    
    coords_list.append(piece)

print coords_list

[array([[  108.,   153.,   107.,   152.],
       [ 4978.,   132.,  4977.,   132.],
       [   77.,  3557.,    78.,  3558.],
       [ 5012.,  3572.,  5012.,  3572.],
       [   82.,  1613.,    82.,  1613.],
       [ 5032.,  1578.,  5032.,  1577.],
       [ 2542.,  3608.,  2542.,  3609.],
       [ 2542.,   123.,  2542.,   123.],
       [ 2492.,  1817.,  2492.,  1817.]])]


In [7]:
coords = np.vstack(coords_list)
print coords

[[  108.   153.   107.   152.]
 [ 4978.   132.  4977.   132.]
 [   77.  3557.    78.  3558.]
 [ 5012.  3572.  5012.  3572.]
 [   82.  1613.    82.  1613.]
 [ 5032.  1578.  5032.  1577.]
 [ 2542.  3608.  2542.  3609.]
 [ 2542.   123.  2542.   123.]
 [ 2492.  1817.  2492.  1817.]]


In [8]:
if zoomed_coords:
    coords /= 5
    coords = coords.round()
    print coords

[[   22.    31.    21.    30.]
 [  996.    26.   995.    26.]
 [   15.   711.    16.   712.]
 [ 1002.   714.  1002.   714.]
 [   16.   323.    16.   323.]
 [ 1006.   316.  1006.   315.]
 [  508.   722.   508.   722.]
 [  508.    25.   508.    25.]
 [  498.   363.   498.   363.]]


In [9]:
lX = coords[:, 0] # leftX coordinates
lY = coords[:, 1] # leftY coordinates
rX = coords[:, 2] # rightX coordinates
rY = coords[:, 3] # rightY coordinates

N = coords.shape[0] # number of pairs of points
M = coords.shape[1] # lX, lY, rX, rY == 4
print N, M

9 4


In [10]:
xi = np.zeros(2*N)

num_coeffs = 6
if consider_distort:
    num_coeffs = 8

print 'Number of coefficients:', num_coeffs
z = np.zeros(num_coeffs)
arr = np.zeros((2*N, num_coeffs)) # matrix A

Number of coefficients: 8


In [11]:
for i in xrange(N): # fill the xi vector
    xi[2*i] = rX[i]
    xi[2*i + 1] = rY[i]

In [12]:
if consider_distort:
    for i in xrange(N): # fill the A matrix
        dist_l = (lX[i]-x_c)**2 + (lY[i]-y_c)**2
        dist_r = (rX[i]-x_c)**2 + (rY[i]-y_c)**2

        zx1 = (lX[i] - x_c) * dist_l
        zx2 = (rX[i] - x_c) * dist_r
        arr[2*i] = [lX[i], lY[i], 0, 0, 1, 0, -zx1, zx2]

        zy1 = (lY[i] - y_c) * dist_l
        zy2 = (rY[i] - y_c) * dist_r
        arr[2*i + 1] = [0, 0, lX[i], lY[i], 0, 1, -zy1, zy2]

else:
    for i in xrange(N): # fill the A matrix
        arr[2*i] = [lX[i], lY[i], 0, 0, 1, 0]

        arr[2*i + 1] = [0, 0, lX[i], lY[i], 0, 1]

In [13]:
print 'A:\n', arr, '\n'
print 'xi:\n', xi

A:
[[  2.200000000000e+01   3.100000000000e+01   0.000000000000e+00
    0.000000000000e+00   1.000000000000e+00   0.000000000000e+00
    1.550267346000e+09  -1.555304959000e+09]
 [  0.000000000000e+00   0.000000000000e+00   2.200000000000e+01
    3.100000000000e+01   0.000000000000e+00   1.000000000000e+00
    1.140266501000e+09  -1.144381914000e+09]
 [  9.960000000000e+02   2.600000000000e+01   0.000000000000e+00
    0.000000000000e+00   1.000000000000e+00   0.000000000000e+00
    1.543774400000e+07  -1.599074500000e+07]
 [  0.000000000000e+00   0.000000000000e+00   9.960000000000e+02
    2.600000000000e+01   0.000000000000e+00   1.000000000000e+00
    4.091002160000e+08  -4.091425100000e+08]
 [  1.500000000000e+01   7.110000000000e+02   0.000000000000e+00
    0.000000000000e+00   1.000000000000e+00   0.000000000000e+00
    1.030521970000e+09  -1.027353600000e+09]
 [  0.000000000000e+00   0.000000000000e+00   1.500000000000e+01
    7.110000000000e+02   0.000000000000e+00   1.000000000

In [14]:
from numpy.linalg import norm, pinv

In [15]:
p_arr = pinv(arr)
z = np.dot(p_arr, xi)
print 'Affine coefficients: \n', z

Affine coefficients: 
[ 0.999775014845  0.000404016568 -0.000473431409  1.000526283842
 -0.160954697816 -0.01320517361   0.000000262007  0.000000261832]


In [16]:
sigma = 2 # 2 pix error
I = np.eye(2*N)

print 'cond(A): ', np.linalg.cond(arr)
# print norm( np.dot(I - np.dot(arr, p_arr), xi) )
proba = (2*N - num_coeffs) / norm( np.dot(I - np.dot(arr, p_arr), xi) )**2
sig_theory = (1 / proba)**0.5

print 'Probability:', proba
print 'Sigma theory:', sig_theory
print 'Sigma practice', sigma

cond(A):  9633046386.47
Probability: 8.81566323639
Sigma theory: 0.336800326812
Sigma practice 2
